In [1]:
!pip install evaluate
!pip install bert_score

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 84.1/84.1 kB 2.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 61.1/61.1 kB 2.4 MB/s eta 0:00:00


In [3]:
import pandas as pd
from evaluate import load
from torch.optim import AdamW
from torch.utils.data import DataLoader
from transformers import AutoTokenizer, AutoModelForSeq2SeqLM, DataCollatorForSeq2Seq
from seq2seq import create_transformers_train_data, train_transformer, decode_with_transformer

In [4]:
data = pd.read_csv('en_es_corpus.txt', sep='\t', header=None)
data = data[[0, 1]]
data.columns = ['EN', 'ES']
data = data[:1000]
sentences_en = data['EN'].values.tolist()
sentences_es = data['ES'].values.tolist()

In [5]:
model_name = 't5-small'

In [6]:
tokenizer = AutoTokenizer.from_pretrained(model_name)
model = AutoModelForSeq2SeqLM.from_pretrained(model_name)

/usr/local/lib/python3.12/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


tokenizer_config.json:   0%|          | 0.00/2.32k [00:00<?, ?B/s]

spiece.model:   0%|          | 0.00/792k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/1.39M [00:00<?, ?B/s]

config.json:   0%|          | 0.00/1.21k [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/242M [00:00<?, ?B/s]

generation_config.json:   0%|          | 0.00/147 [00:00<?, ?B/s]

In [9]:
instruction = 'Translate from English to Spanish: '

In [11]:
sentences_en_with_ins = [f'{instruction}{s}' for s in sentences_en]

In [7]:
sentences_en[0]

'Go.'

In [12]:
sentences_en_with_ins[0]

'Translate from English to Spanish: Go.'

In [8]:
sentences_es[0]

'Ve.'

In [17]:
train_set = create_transformers_train_data(sentences_en_with_ins, sentences_es, tokenizer)
train_set

/usr/local/lib/python3.12/dist-packages/transformers/tokenization_utils_base.py:4118: UserWarning: `as_target_tokenizer` is deprecated and will be removed in v5 of Transformers. You can tokenize your labels by using the argument `text_target` of the regular `__call__` method (either in the same call as your input texts if you use the same keyword arguments, or in a separate call.
  warnings.warn(


Dataset({
    features: ['input_ids', 'attention_mask', 'labels'],
    num_rows: 1000
})

In [18]:
data_collator = DataCollatorForSeq2Seq(tokenizer=tokenizer, model=model)
train_loader = DataLoader(train_set, batch_size=256, shuffle=True, collate_fn=data_collator)

In [15]:
optimizer = AdamW(model.parameters(), lr=0.001)

In [19]:
train_transformer(model, train_loader, optimizer, 5)

Epoch 1/5, Loss: 4.0359
Epoch 2/5, Loss: 3.3469
Epoch 3/5, Loss: 3.0423
Epoch 4/5, Loss: 2.8212
Epoch 5/5, Loss: 2.6366


In [20]:
sentences_en[0]

'Go.'

In [21]:
predicted_sentence = decode_with_transformer(sentences_en_with_ins[0], tokenizer, model)
predicted_sentence

/usr/local/lib/python3.12/dist-packages/transformers/tokenization_utils_base.py:4118: UserWarning: `as_target_tokenizer` is deprecated and will be removed in v5 of Transformers. You can tokenize your labels by using the argument `text_target` of the regular `__call__` method (either in the same call as your input texts if you use the same keyword arguments, or in a separate call.
  warnings.warn(


'Vete.'

In [22]:
reference_sentence = sentences_es[0]
reference_sentence

'Ve.'

In [23]:
bleu = load('bleu')

In [24]:
bleu.compute(predictions=[predicted_sentence], references=[reference_sentence])

{'bleu': 0.0,
 'precisions': [0.5, 0.0, 0.0, 0.0],
 'brevity_penalty': 1.0,
 'length_ratio': 1.0,
 'translation_length': 2,
 'reference_length': 2}

In [25]:
bertscore = load('bertscore')

In [26]:
bertscore.compute(predictions=[predicted_sentence], references=[reference_sentence], model_type='microsoft/deberta-xlarge-mnli')

tokenizer_config.json:   0%|          | 0.00/52.0 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/792 [00:00<?, ?B/s]

vocab.json: 0.00B [00:00, ?B/s]

merges.txt: 0.00B [00:00, ?B/s]

pytorch_model.bin:   0%|          | 0.00/3.04G [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/3.04G [00:00<?, ?B/s]

{'precision': [0.7263869047164917],
 'recall': [0.7734212875366211],
 'f1': [0.7491666078567505],
 'hashcode': 'microsoft/deberta-xlarge-mnli_L40_no-idf_version=0.3.12(hug_trans=4.57.2)'}